In [2]:
from opentrons import robot, containers, instruments
import pandas as pd
import argparse
import sys

In [3]:
# Load our files
plates = pd.read_csv('../synth2/Twist_10-26-2017_648.csv',usecols=['Plate','Well','customer_line_item_id','synthesized sequence length','Yield (ng)'])

#layout = pd.read_csv('./testing/plate-loc-resuspend-sm.csv')
#layout = layout.set_index('Position').to_dict()['Name'] # Turn into a location->name dict


In [4]:
# Configuration
SOURCE_SLOTS = ['D1','D2','D3', 'E1','B2']

plate_names = plates['Plate'].unique()

if len(plate_names) > len(SOURCE_SLOTS):
    print("Error: This build plan requires too many source plates.")
    sys.exit(1)

layout = list(zip(SOURCE_SLOTS[:len(plate_names)], plate_names))

slots = pd.Series(SOURCE_SLOTS)
columns = sorted(slots.str[0].unique())
rows = sorted(slots.str[1].unique(), reverse=True)

layout_table = pd.DataFrame(index=rows, columns=columns)
layout_table.fillna("", inplace=True)

for slot, plate in layout:
    layout_table.loc[slot[1], slot[0]] = plate

print("Please arrange the plates in the following configuration:")
print()
print(layout_table)
print()
input("Press enter to continue")

layout = dict(layout)

Please arrange the plates in the following configuration:

  B                   D E
3                        
2                        
1    pSHPs1025B525648MU  

Press enter to continue


In [7]:
# Calculate Resuspension Volumes

plates['customer_line_item_id'] = plates['customer_line_item_id'].str.strip()

amount = plates['Yield (ng)']
length = plates['synthesized sequence length']

fmoles = 40

volume = (((((amount / 1000)*(1000000))/(660*length))*1000) / fmoles) * 2

plan = pd.DataFrame({'Plate': plates['Plate'],
                     'Well': plates['Well'],
                     'Volume': volume})
plan = plan[['Plate','Well','Volume']]


In [47]:
# Configure the robot

#  Layout:
#    A     B       C      D      E
#  3 trash master  source source p10
#  2 p200  dest    source source p10
#  1 p200  dest    source source p10

robot.connect('Virtual Smoothie') #robot.get_serial_ports_list()[0])



In [9]:
dest_plates = [
    containers.load('96-flat', 'B1'),
    containers.load('96-flat', 'B2')
]

source_plates = {}
for slot, plate in layout.items():
    source_plates[plate] = containers.load('96-flat', slot)


In [21]:
# Distribute down the plates
num_reactions = len(plan)
num_rows = num_reactions // 8 + 1
all_wells = dest_plates[0].wells() + dest_plates[1].wells()

all_wells[:num_reactions]

<WellSeries: <Well A1><Well B1><Well C1><Well D1><Well E1><Well F1><Well G1><Well H1><Well A2><Well B2><Well C2><Well D2><Well E2><Well F2><Well G2><Well H2><Well A3><Well B3><Well C3><Well D3><Well E3><Well F3>>

In [70]:
import numpy as np


let_num = {
    "A" : 0,
    "B" : 1,
    "C" : 2,
    "D" : 3,
    "E" : 4
}
tip_rack1 = "E2"
tip_rack2 = "E3"
#print(let_num[tip_rack1[0]])

#layout = np.matrix([[" ","A","B","C","D","E"],
#         ["1"," ","p200"," "," ","p10"],
#         ["2"," "," "," "," "," "],
#         ["3"," "," "," "," "," "]])

layout = np.matrix([["   A3   ","   B3   ","   C3   ","   D3   ","   E3   "],
         ["   A2   ","   B2   ","   C2   ","   D2   ","   E2   "],
         ["   A1   ","   B1   ","   C1   ","   D1   ","   E1   "]])

print(layout)
locations = {
    "10r___1" : "D3",
    "10r___2" : "B2",
    "200r__1" : "C1"
}

for item in locations:
    print(item)
    print(locations[item])
    slot = locations[item]
    print("slot[0]",slot[0])
    row = int(let_num[slot[0]])
    print("row", row)
    col = 3 - int(slot[1])
    print("col", col)
    print(layout[col,row])
    layout[col,row] = item

#tip_rack1 = "E2"
#tip_rack2 = "E3"
#print(layout[2,4])
#print(layout[tip_rack1[1],let_num[tip_rack1[0]]])
#layout[let_num[let_num[tip_rack1[0]]],]
print(layout)





[['   A3   ' '   B3   ' '   C3   ' '   D3   ' '   E3   ']
 ['   A2   ' '   B2   ' '   C2   ' '   D2   ' '   E2   ']
 ['   A1   ' '   B1   ' '   C1   ' '   D1   ' '   E1   ']]
10r__1
D3
slot[0] D
row 3
col 0
   D3   
10r__2
B2
slot[0] B
row 1
col 1
   B2   
200r_1
C1
slot[0] C
row 2
col 2
   C1   
[['   A3   ' '   B3   ' '   C3   ' '10r__1' '   E3   ']
 ['   A2   ' '10r__2' '   C2   ' '   D2   ' '   E2   ']
 ['   A1   ' '   B1   ' '200r_1' '   D1   ' '   E1   ']]


In [14]:
df =pd.DataFrame(index=range(1,4), columns=['A','B','C'])
df.loc[1,'A'] = 'hi'
df

,A,B,C
1,hi,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN


In [20]:
import numpy as np
import pandas as pd

locations = np.array([["tiprack-200", "A3"],
                    ["tiprack-10", "E3"],
                    ["tiprack-10s1", "E2"],
                    ["tiprack-10s2", "E1"],
                    ["trash", "B1"],
                    ["PCR-strip-tall", "C3"],
                    ["resuspension-plate", "C2"]])

locations_df = pd.DataFrame({"Item" : locations[:,0],
                        "Location" : locations[:,1]})

#layout_table =pd.DataFrame(index=[3,2,1], columns=['A','B','C','D','E'])

deck = ['A1','B2','C3','D2','E1']
slots = pd.Series(deck)
columns = sorted(slots.str[0].unique())
rows = sorted(slots.str[1].unique(), reverse=True)
layout_table = pd.DataFrame(index=rows, columns=columns)
layout_table.fillna("", inplace=True)

for row,col in locations:
    layout_table.loc[col[1], col[0]] = row

print("Please arrange the plates in the following configuration:")
print()
print(layout_table)
print()
#input("Press enter to continue")

Please arrange the plates in the following configuration:

             A      B                   C D             E
3  tiprack-200             PCR-strip-tall      tiprack-10
2                      resuspension-plate    tiprack-10s1
1               trash                        tiprack-10s2



In [10]:
import numpy as np
import pandas as pd

#dates = ["2000-01-01","2000-01-01","2000-01-01","2000-01-01","2000-01-02","2000-01-02","2000-01-02","2000-01-02"]
#df = pd.DataFrame(np.random.randn(8, 4), index=dates, columns=['A', 'B', 'C', 'D'])
#print(df)
#print(df.loc["2000-01-01"])

plates = pd.read_csv("../plate_maps/Plates 37-40 unmodified.csv")
plates = plates.set_index('Plate')
plates.loc["pSHPs0807B412037MU"]

,Well,customer_line_item_id,line_item_number,order_item_id,Insert Sequence,Insert Length,Frag Analysis Status,synthesized sequence length,Yield (ng)
Plate,,,,,,,,,
pSHPs0807B412037MU,A1,\tMMSYN1_0002_1,1,OI_5977507a77383b6ba724fdc0,GAAGACATAATGAACTTCTCTATTAATCGTATGGTTTTGTTAGATA...,1146,Pass,1189,200
pSHPs0807B412037MU,B1,\tMMSYN1_0821_1,2,OI_5977507a77383b6ba724fdc1,GAAGACATAATGGAGAAGTTCACTATCAAGGATTTAACGGACAACT...,960,Pass,1003,200
pSHPs0807B412037MU,C1,\tMMSYN1_0329_1,4,OI_5977507a77383b6ba724fdc3,GAAGACATAATGTCGCTGGAGCGTTTGCAAAAGGTAATCAGTAGCC...,777,Pass,820,200
pSHPs0807B412037MU,D1,\tMMSYN1_0027_1,5,OI_5977507a77383b6ba724fdc4,GAAGACATAATGATTAAAAAGTATGAGGTGATGTATATTTTGGATC...,432,Pass,475,200
pSHPs0807B412037MU,E1,\tMMSYN1_0378_1,6,OI_5977507a77383b6ba724fdc5,GAAGACATAATGCAGACCAACCTGAAACAGTATCTGGATTATCTGG...,756,Pass,799,200
pSHPs0807B412037MU,F1,\tMMSYN1_0515_1,7,OI_5977507a77383b6ba724fdc6,GAAGACATAATGTCAAAACGCCTTGATTATCTTAGCTGGCAACATT...,501,Pass,544,200
pSHPs0807B412037MU,G1,\tMMSYN1_0800_1,9,OI_5977507a77383b6ba724fdc8,GAAGACATAATGTCGAACCGCATTTACATTGGTAATGACCATTCGG...,462,Pass,505,200
pSHPs0807B412037MU,H1,\tMMSYN1_0380_1,10,OI_5977507a77383b6ba724fdc9,GAAGACATAATGAGCAAGAAAATTGCTTTGTTTGGTGGCTCCTTCG...,1116,Pass,1159,200
pSHPs0807B412037MU,A2,\tMMSYN1_0402_1,11,OI_5977507a77383b6ba724fdca,GAAGACATAATGATTCGCATTAATTATTTCAATGAAACGGATATTA...,513,Pass,556,200
